<a href="https://colab.research.google.com/github/BelemoualemChaimae/NLP/blob/main/information-retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz
	
!gzip -d msmarco-docs.tsv.gz
!gzip -d msmarco-doctrain-queries.tsv.gz
!gzip -d msmarco-doctrain-top100.gz


--2021-02-02 20:14:11--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 40.112.152.16
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|40.112.152.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8446274598 (7.9G) [application/x-gzip]
Saving to: ‘msmarco-docs.tsv.gz’

msmarco-docs.tsv.gz 100%[===================>]   7.87G  14.9MB/s    in 11m 25s 

2021-02-02 20:25:37 (11.8 MB/s) - ‘msmarco-docs.tsv.gz’ saved [8446274598/8446274598]

--2021-02-02 20:25:37--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 40.112.152.16
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|40.112.152.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6457962 (6.2M) [application/x-gzip]
Saving to

In [ ]:
!gzip -d msmarco-docs.tsv.gz

gzip: msmarco-docs.tsv.gz: No such file or directory


In [ ]:
!pip install dask 

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
queries_train=pd.read_table('msmarco-doctrain-queries.tsv',header=None)
queries_train.columns=['qid','query']
print('Shape=>',queries_train.shape)
queries_train.head()

Shape=> (367013, 2)


,qid,query
0,1185869,)what was the immediate impact of the success ...
1,1185868,_________ justice is designed to repair the ha...
2,1183785,elegxo meaning
3,645590,what does physical medicine do
4,186154,feeding rice cereal how many times per day


In [ ]:
queries=queries_train.sample(n=2000,random_state=42).reset_index(drop=True)
print('Shape=>',queries.shape)
queries.head()

Shape=> (2000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# Creating Training Set of Queries
training_queries=queries.iloc[:1000]
print('Shape=>',training_queries.shape)
training_queries.head()

Shape=> (1000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# Creating Testing Set of Queries
testing_queries=queries.iloc[1000:]
print('Shape=>',testing_queries.shape)
testing_queries.head()

Shape=> (1000, 2)


,qid,query
1000,807599,what is the axis mundi?
1001,990945,where is pratt kansas
1002,48210,backordered definition
1003,894254,what show did simon baker play in
1004,165579,does dna replication in mitosis and meiosis


In [ ]:
train_top100=pd.read_table('/content/msmarco-doctrain-top100',delimiter=' ',header=None)
train_top100.columns=['qid','Q0','docid','rank','score','runstring']
print('Shape=>',train_top100.shape)
train_top100.head()

Shape=> (36701116, 6)


,qid,Q0,docid,rank,score,runstring
0,1185869,Q0,D59221,1,-4.80433,IndriQueryLikelihood
1,1185869,Q0,D59220,2,-4.92127,IndriQueryLikelihood
2,1185869,Q0,D2192591,3,-5.05215,IndriQueryLikelihood
3,1185869,Q0,D2777518,4,-5.05486,IndriQueryLikelihood
4,1185869,Q0,D2371978,5,-5.07048,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for training
training_ranked100=train_top100[train_top100['qid'].isin(training_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',training_ranked100.shape)
training_ranked100.head()

Shape=> (100000, 6)


,qid,Q0,docid,rank,score,runstring
0,310290,Q0,D579750,1,-5.11498,IndriQueryLikelihood
1,310290,Q0,D579754,2,-5.57703,IndriQueryLikelihood
2,310290,Q0,D2380815,3,-5.84852,IndriQueryLikelihood
3,310290,Q0,D822566,4,-5.95002,IndriQueryLikelihood
4,310290,Q0,D2249695,5,-6.08326,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for testing
testing_ranked100=train_top100[train_top100['qid'].isin(testing_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',testing_ranked100.shape)
testing_ranked100.head()

Shape=> (100000, 6)


,qid,Q0,docid,rank,score,runstring
0,1164761,Q0,D3261512,1,-4.92320,IndriQueryLikelihood
1,1164761,Q0,D1529569,2,-5.01292,IndriQueryLikelihood
2,1164761,Q0,D3444265,3,-5.03616,IndriQueryLikelihood
3,1164761,Q0,D1313045,4,-5.09482,IndriQueryLikelihood
4,1164761,Q0,D1058999,5,-5.18936,IndriQueryLikelihood


In [ ]:
# Labelling Top 10 as 1 and last 10 as 0
rel=list(range(1,11))
nonrel=list(range(91,101))
training_ranked100['rel']=training_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))
testing_ranked100['rel']=testing_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))

# Result set for Training
training_result=training_ranked100.dropna()
training_result['rel']=training_result['rel'].astype(int)
print('Shape=>',training_result.shape)
training_result.head()


Shape=> (20000, 7)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,qid,Q0,docid,rank,score,runstring,rel
0,310290,Q0,D579750,1,-5.11498,IndriQueryLikelihood,1
1,310290,Q0,D579754,2,-5.57703,IndriQueryLikelihood,1
2,310290,Q0,D2380815,3,-5.84852,IndriQueryLikelihood,1
3,310290,Q0,D822566,4,-5.95002,IndriQueryLikelihood,1
4,310290,Q0,D2249695,5,-6.08326,IndriQueryLikelihood,1


In [ ]:
# Result set for Testing
testing_result=testing_ranked100.dropna()
testing_result['rel']=testing_result['rel'].astype(int)
print('Shape=>',testing_result.shape)
testing_result.head()

Shape=> (20000, 7)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,qid,Q0,docid,rank,score,runstring,rel
0,1164761,Q0,D3261512,1,-4.92320,IndriQueryLikelihood,1
1,1164761,Q0,D1529569,2,-5.01292,IndriQueryLikelihood,1
2,1164761,Q0,D3444265,3,-5.03616,IndriQueryLikelihood,1
3,1164761,Q0,D1313045,4,-5.09482,IndriQueryLikelihood,1
4,1164761,Q0,D1058999,5,-5.18936,IndriQueryLikelihood,1


In [ ]:
!pip install dask[dataframe]

     |████████████████████████████████| 102kB 6.9MB/s 


In [ ]:
import dask.dataframe as dd


In [ ]:
!pip install dask[complete]

     |████████████████████████████████| 675kB 7.6MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=57f592e0d000353279efb906638bb7fd08d5ed3263e41dcea64e65e8683f8f6c
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
ERROR: distributed 2021.1.1 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [ ]:
df=dd.read_table('/content/msmarco-docs.tsv',header=None)
df.columns=['docid','url','title','body']
df.head()

,docid,url,title,body
0,D1555982,https://answers.yahoo.com/question/index?qid=2...,The hot glowing surfaces of stars emit energy ...,Science & Mathematics Physics The hot glowing ...
1,D301595,http://childparenting.about.com/od/physicalemo...,Developmental Milestones and Your 8-Year-Old C...,School-Age Kids Growth & Development Developme...
2,D1359209,http://visihow.com/Check_for_Lice_Nits,Check for Lice Nits,Check for Lice Nits Edited by Mian Sheilette O...
3,D2147834,http://www.nytimes.com/2010/01/05/business/glo...,Dubai Opens a Tower to Beat All,Global Business Dubai Opens a Tower to Beat Al...
4,D1568809,http://www.realtor.com/realestateandhomes-sear...,"Coulterville, CA Real Estate & Homes for Sale","Coulterville, CA Real Estate & Homes for Sale4..."


In [ ]:
def create_corpus(result):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  print('Number of Rows=>',len(corpus))
  return corpus

training_corpus=create_corpus(training_result)
training_corpus.head()

Number of Rows=> 19505


,docid,title,body
0,D297612,"Fair Oaks, CA County Of Sacramento","Home Fair Oaks, CA County Of Sacramento Fair O..."
1,D1036761,What airport is the closest to downtown London?,Answers.com ® Wiki Answers ® Categories Travel...
2,D2025493,"York County, South Carolina Genealogy",navigation search United States South Carolina...
3,D2214523,The Natural Habitat of Wolves,"Wolves are members of the canine family, but t..."
4,D1881859,Franking Privilege Law and Legal Definition,Franking Privilege Law and Legal Definition Fr...


In [ ]:
testing_corpus=create_corpus(testing_result)
testing_corpus.head()

Number of Rows=> 19570


,docid,title,body
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...
1,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...
2,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
3,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...
4,D2078142,What are all the literary devices? List them p...,Education & Reference Homework Help What are a...


In [ ]:
temp_doc=training_corpus.partitions[1]
temp_doc=temp_doc.compute()
temp_doc=temp_doc.sample(1)
print('Title \n',temp_doc['title'].values[0])
print('Body:\n',temp_doc.body.values[0])

Title 
 .
Body:
 View Location Examples Your local forecast office is Binghamton, NYNews Headlines SKYWARN (Weather Spotters) Spring Training schedule!What we are talking about. Additional Headlines Hazardous Weather Conditions Hazardous Weather Outlook En Español Share |Current conditions at Cortland County-Chase Field (KN03)Lat: 42.59°N Lon: 76.21°W Elev: 1198ft. Fog/Mist27°F-3°CHumidity 100%Wind Speed Calm Barometer 30.00 in Dewpoint 27°F (-3°C)Visibility 5.00 mi Last update 12 Apr 4:15 am EDTMore Information: Local Forecast Office More Local Wx3 Day History Mobile Weather Hourly Weather Forecast Extended Forecast for Cortland NYToday Rain Likely High: 50 °FTonight Chance Rain then Mostly Cloudy Low: 42 °FFriday Chance Showers High: 60 °FFriday Night Slight Chance Showers Low: 44 °FSaturday Chance Showers High: 54 °FSaturday Night Showers Likely Low: 39 °FSunday Showers Likely High: 54 °FSunday Night Showers Low: 43 °FMonday Showers High: 49 °FDetailed Forecast Today Rain likely, ma

In [ ]:
temp_doc.head()

,docid,title,body
22,D508614,.,View Location Examples Your local forecast off...


In [ ]:
temp_doc.head()

,docid,title,body
22,D508614,.,View Location Examples Your local forecast off...


In [ ]:
#Pre processing sreps 

import re

# Lowercasing the text
training_corpus['cleaned']=training_corpus['body'].apply(lambda x:x.lower(), meta=('Object'))
testing_corpus['cleaned']=testing_corpus['body'].apply(lambda x:x.lower(), meta=('Object'))

In [ ]:
tet=training_corpus.partitions[1]
tet=tet.compute()
tet=tet.sample(1)
print('ClenedBody=>',tet['cleaned'].values[0])
print('IS THERE ANY # ------- Let Check ! ')
print('Body:\n',tet.body.values[0])

ClenedBody=> from wikipedia, the free encyclopedianavigation search [ hide]this article has multiple issues. please help improve it or discuss these issues on the talk page. ( learn how and when to remove these template messages)this article may contain an excessive amount of intricate detail that may only interest a specific audience. (july 2016)this article describes a work or element of fiction in a primarily in-universe style. (july 2016)in j. k. rowling 's harry potter series, magic is depicted as a supernatural force that can be used to override the usual laws of nature. many fictional magical creatures exist in the series, while ordinary creatures sometimes exhibit new magical properties in the novel's world. objects, too, can be enhanced or imbued with magical property. the small percentage of humans who are able to perform magic are referred to as witches and wizards, in contrast to the non-magical muggles. in humans, magic or the lack thereof is an inborn attribute. it is inh

In [ ]:
# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x:expand_contractions(x), meta=('Object'))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x:expand_contractions(x), meta=('Object'))

In [ ]:
# Function for Cleaning Text
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text
 
# Cleaning corpus using RegEx
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x: clean_text(x), meta=('Object'))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x: clean_text(x), meta=('Object'))

In [ ]:
# Removing extra spaces
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x), meta=('Object'))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x), meta=('Object'))

In [ ]:
tet=training_corpus.partitions[1]
tet=tet.compute()
tet=tet.sample(1)
print('ClenedBody: \n',tet['cleaned'].values[0])
print('IS THERE ANY # ------- Let Check ! The Second Time Yep Chama')
print('Body:\n',tet.body.values[0])

ClenedBody: 
 caldwell county texas from wikipedia the free encyclopedianavigation search not to be confused with caldwell texas caldwell county texas the caldwell county courthouse in lockhart location in the u s state of texas texas is location in the u s founded for mathew caldwell seat lockhart largest city lockhart area total sq mi km land sq mi km water sq mi km population density sq mi km congressional districts time zone central utc co caldwell tx us caldwell county is a county in the u s state of texas as of the census the population was its county seat is lockhart the county was founded in and named after mathew caldwell a ranger captain who fought in the battle of plum creek against the comanches and against santa anna is armies during the texas revolution caldwell was also a signer of the texas declaration of independence caldwell county is part of the greater austin metropolitan area major adjacent unincorporated ghost see external links history edit this section is in a l

In [ ]:
# Stopwords removal & Lemmatizing tokens using SpaCy
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000

# Removing Stopwords and Lemmatizing words
training_corpus['lemmatized']=training_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]), meta=('Object'))
testing_corpus['lemmatized']=testing_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]), meta=('Object'))


In [ ]:
tet=training_corpus.partitions[1]
tet=tet.compute()
tet=tet.sample(1)
print('Lemmatization: \n',tet['lemmatized'].values)
print('IS THERE ANY # ------- Let Check ! The Second Time Yep Chama')
print('Body:\n',tet.body.values[0])

Lemmatization: 
 ['thyroid cancer diagnosis test exam diagnose thyroid cancer write bridget brady md facsin order accurately diagnose treat thyroid cancer doctor thoroughly review complete medical history include information family member thyroid cancer benign tumor multiple endocrine neoplasia doctor perform physical examination order test blood work physical exam doctor palpate feel neck require swallow flex bend neck pay particular attention thyroid gland surround tissue lymph nod number size shape firmness nodule s carefully examine doctor correlate physical finding medical history report symptom pain hoarseness fact american thyroid association report thyroid lump nodule common few cancer history thyroid cancer diagnosis family history medullary thyroid cancer doctor test blood calcitonin calcium level calcitonin hormone important calcium phosphorus metabolism bone growth elevated calcitonin level indicate cancer laboratory test diagnose thyroid cancer blood draw test thyroid glan

In [ ]:
# Lowercasing the text
training_queries['cleaned']=training_queries['query'].apply(lambda x:x.lower())
testing_queries['cleaned']=testing_queries['query'].apply(lambda x:x.lower())

# Expanding contractions
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x:expand_contractions(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x:expand_contractions(x))

# Cleaning queries using RegEx
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: clean_text(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: clean_text(x))

# Removing extra spaces
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
training_queries

,qid,query,cleaned
0,687888,what is a jpe,what is a jpe
1,480210,price for asphalt driveway,price for asphalt driveway
2,591004,what causes pressure skin bruising,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...,average number of bowel movements per day for ...
...,...,...,...
995,89597,cell voltage mv meaning,cell voltage mv meaning
996,1167043,what an ip address,what an ip address
997,737304,what is daily max citizens atm,what is daily max citizens atm
998,156934,do i need a florida commercial driver license,do i need a florida commercial driver license


In [ ]:
combined_training=dd.concat([training_corpus.rename(columns={'lemmatized':'text'})['text'],\
                             training_queries.rename(columns={'cleaned':'text'})['text']])\
                             .sample(frac=1).reset_index(drop=True)

In [ ]:
combined_training.size.compute()

20505

In [ ]:
from gensim.models import Word2Vec

# Creating data for the model training
train_data=[]
for i in combined_training:
    train_data.append(i.split())

# Training a word2vec model from the given data set
w2v_model = Word2Vec(train_data, size=300, min_count=2,window=5, sg=1,workers=4)

In [ ]:
# Vocabulary size
print('Vocabulary size:', len(w2v_model.wv.vocab))

Vocabulary size: 198281


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.vocab:
                embeddings.append(w2v_model.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()),meta=("Object"))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
######################################################tHIS TAKES TOO MUCH TIME TO RUN 
# Function for calculating average precision for a query
def average_precision(qid,qvector):
  
  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=dd.merge(qresult,qcorpus,on='docid')

  
  # Ranking documents for the query
  
  
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item(),meta=("Object"))
  qresult['similarity']=qresult['similarity'].astype("float")
  ranking=qresult.nlargest(n=10,columns ='similarity')
  
  #ranking.head()
  # Taking Top 10 documents for the evaluation
  ranked=ranking.head(10)['rel'].values
  
  # Calculating precision

  precision=[]
  for i in range(1,11):
    if ranked[i-1]:
      precision.append(np.sum(ranked[:i])/i)
  
  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

KeyboardInterrupt: ignored

In [ ]:
def ranking_ir(query):
  
  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  vector=get_embedding_w2v(query.split())

  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item(),meta=('Object'))
  #documents.sort_values(by='similarity',ascending=False,inplace=True)
 
  return documents.nlargest(n=10,columns ='similarity').head(10)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity
11,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.858728
34,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.792597
1,D2218024,What draft round was Michael Jordan picked?,Answers.com ® Wiki Answers ® Categories Entert...,0.783932
28,D1656700,What nba team did Michael Jordan get drafted to?,What nba team did Michael Jordan get drafted t...,0.782568
27,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.780309
28,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.768375
23,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.766328
27,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.765038
24,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.760559
42,D1900195,What year did Michael Jordan play baseball?,Answers.com ® Wiki Answers ® Categories Entert...,0.754815
